In [88]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Source donnée : https://www.ncei.noaa.gov/cdo-web/customoptions

PRCP = précipitation

SNWD = épaisseur de neige (snow depth)

TMAX = température maximale

NAME = nom de la station

In [ ]:
weather = pd.read_csv("data/weather.csv")
weather = weather.loc[weather["NAME"] == "PARIS MONTSOURIS, FR"]
weather["SNWD"] = weather["SNWD"].replace(np.NaN, 0.)
weather["DATE"] = pd.to_datetime(weather["DATE"])

#Remplacer les nans de TMAX à partir des valeurs de TMIN et TAVG
weather["TMAX Backup"] = 2*weather["TAVG"] - weather["TMIN"]
weather["TMAX"] = np.where(weather["TMAX"] == weather["TMAX"], weather["TMAX"], weather["TMAX Backup"])
weather["TMAX Backup 2"] = weather["TAVG"] * np.mean(weather["TMAX"]/weather["TAVG"])
weather["TMAX"] = np.where(weather["TMAX"] == weather["TMAX"], weather["TMAX"], weather["TMAX Backup 2"])

weather = weather.drop(columns=["STATION", "LATITUDE", "LONGITUDE", "ELEVATION", "PRCP_ATTRIBUTES", "SNWD_ATTRIBUTES", "TAVG_ATTRIBUTES", "TMAX_ATTRIBUTES", "TMIN_ATTRIBUTES", "TMAX Backup", "TMAX Backup 2", "TAVG", "TMIN"])
weather

,NAME,DATE,PRCP,SNWD,TMAX
0,"PARIS MONTSOURIS, FR",2010-01-01,0.01,0.0,36.0
1,"PARIS MONTSOURIS, FR",2010-01-02,0.00,0.0,38.0
2,"PARIS MONTSOURIS, FR",2010-01-03,0.00,0.0,40.0
3,"PARIS MONTSOURIS, FR",2010-01-04,0.00,0.0,33.0
4,"PARIS MONTSOURIS, FR",2010-01-05,0.00,0.0,33.0
...,...,...,...,...,...
5405,"PARIS MONTSOURIS, FR",2024-11-23,0.01,0.4,50.0
5406,"PARIS MONTSOURIS, FR",2024-11-24,0.00,0.0,63.0
5407,"PARIS MONTSOURIS, FR",2024-11-25,0.02,0.0,63.0
5408,"PARIS MONTSOURIS, FR",2024-11-26,0.44,0.0,51.0


In [87]:
weather.to_csv("data/cleaned_weather.csv")